## This notebook is used to generate the data for MLE estimation

In [1]:
import numpy as np
from scipy.stats import norm

In [2]:
# Define the parameters
factor_loading = 0.45
gamma = -1.5

In [3]:
# Generate normalized return on obligors’ assets (Y) for only 1 bucket
def generate_y(factor_loading, num_of_obligors):
    x = np.random.normal()
    epsilon = np.random.normal(0, 1, num_of_obligors)
    y = factor_loading * x + epsilon * (1 - factor_loading**2)**0.5
    return y

In [4]:
generate_y(factor_loading, 5)

array([ 0.72011136,  1.3876965 ,  0.7054479 ,  1.61423285, -0.05650462])

In [5]:
def generate_default(num_of_obligors, factor_loading, gamma):
    y = generate_y(factor_loading, num_of_obligors)
    default = (y < gamma).sum()
    return default

In [6]:
n_g = 10000
d_g = generate_default(n_g, factor_loading, gamma)
print("Number of defaults: ", d_g)

Number of defaults:  2782


In [7]:
norm.cdf(gamma) * n_g

668.0720126885806

In [8]:
default_list = []
for i in range(100):
    default_list.append(generate_default(n_g, factor_loading, gamma))
    
np.mean(default_list)

632.31

In [9]:
# Generate normalized return on obligors’ assets (Y) for more buckets
def generate_default_buckets(factor_loading_list, num_of_obligors_list, gamma_list):
    x = np.random.normal()
    defaults_list = []
    
    for index, num_of_obligors in enumerate(num_of_obligors_list):
        epsilon = np.random.normal(0, 1, num_of_obligors)
        y = factor_loading_list[index] * x + epsilon * (1 - factor_loading_list[index]**2)**0.5
        defaults_list.append((y < gamma_list[index]).sum())

    return defaults_list

In [10]:
factor_loading_list = [0.45, 0.5, 0.55]
num_of_obligors_list = [10000, 5000, 20000]
gamma_list = [-1.5, -2, -1.4]
n_g_list = generate_default_buckets(factor_loading_list,num_of_obligors_list, gamma_list)
print("Number of defaults: ", n_g_list)

Number of defaults:  [2895, 697, 8298]
